In [1]:
import pandas as pd, numpy as np,xarray as xr
from pathlib import Path
import re, yaml, copy

In [2]:
# base = Path("/home/julienb/Documents/database_scripts/database_scripts_test/poly_dat_files/Rats/Test_Julien_ForcedInput/TEST_FORCED_INPUT_01")
base = Path("/home/julienb/Documents/database_scripts/database_scripts_test/poly_dat_files/Rats/Luisa/Rat101_0729_opto_01")
dat_path = base.with_suffix(".dat")
task_path = base.with_suffix(".xls")
info_path = base.with_suffix(".yaml")
exists = {"dat_path":dat_path.exists(), "task_path":task_path.exists(), "info_path": info_path.exists()}
if not np.all(list(exists.values())):
    display(exists)
    raise Exception("Missing some input files...")

In [3]:
event_df = pd.read_csv(dat_path, sep="\t", names=['time (ms)', 'family', 'nbre', '_P', '_V', '_L', '_R', '_T', '_W', '_X', '_Y', '_Z'], skiprows=13, dtype=int)
event_df.insert(0, "t", event_df.pop("time (ms)")/1000)
event_df = event_df.reset_index(names="poly_evnum").sort_values(["t", "poly_evnum"]).reset_index(drop=True)
event_df["task_node"] = event_df["_T"].where(event_df["family"]==10).ffill()
print(event_df.to_string())

      poly_evnum         t  family  nbre  _P  _V  _L  _R   _T   _W   _X   _Y  _Z  task_node
0              0     0.006      10     1   1   1   0   0    1    0    0    0   0        1.0
1              1     0.023      10     1   1   1   0   0   36    0    0    0   0       36.0
2              2     0.023       6    22   1   1   1   0    2    1    1    1   0       36.0
3              3     0.118      10     1   1   1   0   0   38    0    0    0   0       38.0
4              4     0.128      10     1   1   1   0   0    2    0    0    0   0        2.0
5              5     0.128      10     1   1   1   0   0    3    0    0    0   0        3.0
6              6     0.128       1     4   0   0   0   0    0    0    0    0   0        3.0
7              7     0.128       1     3   0   0   0   0    0    0    0    0   0        3.0
8              8     0.128       1     2   0   0   0   0    0    0    0    0   0        3.0
9              9     0.128       1     4   0   0   0   0    0    0    0    0   0

In [4]:
task_df = pd.read_csv(task_path, sep="\t", header=11)
task_df = task_df.rename(columns={task_df.columns[0]: "task_node" })
display(task_df.columns)
task_df

Index(['task_node', 'T1', 'T2', 'T3', 'T4', 'LED1(1,2)', 'LED2(1,3)',
       'LED3(1,4)', 'L1(2,1)', 'L2(2,2)', 'LK1(5,1)', 'LK2(5,2)', 'RD(6,11)',
       'ASND(6,20)', 'PAD(6,22)', 'STR1(13,1)', 'STR2(13,2)', 'TTLP1(15,1)',
       'TTLP2(15,2)', 'TTLP3(15,3)', 'TTLP4(15,4)', 'TTLP5(15,5)',
       'TTLP6(15,6)', 'TTLP7(15,7)', 'TTLP8(15,8)', 'NEXT1', 'NEXT2', 'NEXT3',
       'CT1', 'CT2'],
      dtype='object')

,task_node,T1,T2,T3,T4,"LED1(1,2)","LED2(1,3)","LED3(1,4)","L1(2,1)","L2(2,2)",...,"TTLP4(15,4)","TTLP5(15,5)","TTLP6(15,6)","TTLP7(15,7)","TTLP8(15,8)",NEXT1,NEXT2,NEXT3,CT1,CT2
0,1,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,!ti(36),NaN,NaN,NaN,NaN
1,2,_essais,NaN,NaN,NaN,!on,!on,!on,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,PAD_L=1,L1_L=1(28),L2_L=1(26),NaN,NaN
2,3,70,NaN,NaN,NaN,!on,!on,!on,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,PAD_L=0(2),!ti,NaN,NaN,NaN
3,4,100,NaN,NaN,NaN,NaN,NaN,on,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,PAD_L=0(2),!ti,NaN,NaN,NaN
4,5,400-1400,NaN,NaN,NaN,NaN,NaN,on,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,PAD_L=0(2),!ti,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61,62,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
62,63,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
63,64,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
64,65,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:


channels = pd.Series(task_df.columns).str.extract(r'\s*(?P<channel_name>\w+)\s*\((?P<family>\d+)\s*,\s*(?P<nbre>\d+)\)\s*').assign(taskcol_name=task_df.columns).dropna(how="any")
channels["family"] = channels["family"].astype(int)
channels["nbre"] = channels["nbre"].astype(int)
channels

    

,channel_name,family,nbre,taskcol_name
5,LED1,1,2,"LED1(1,2)"
6,LED2,1,3,"LED2(1,3)"
7,LED3,1,4,"LED3(1,4)"
8,L1,2,1,"L1(2,1)"
9,L2,2,2,"L2(2,2)"
10,LK1,5,1,"LK1(5,1)"
11,LK2,5,2,"LK2(5,2)"
12,RD,6,11,"RD(6,11)"
13,ASND,6,20,"ASND(6,20)"
14,PAD,6,22,"PAD(6,22)"


In [6]:
pattern=r'on\(\d+(,\d+)*\)'
task_info=pd.DataFrame()
task_info["data"] = task_df.set_index("task_node")[channels["taskcol_name"].to_list()].stack().str.lower().str.strip()
task_info.index.names=["task_node", "taskcol_name"]
task_info["match"] = task_info["data"].str.fullmatch(pattern)
task_info = task_info.loc[task_info["match"]]
task_info["important"] = task_info["data"].str.slice(3, -1)
task_info["task_params"] = task_info["important"].str.split(",").apply(lambda l: [float(x) for x in l])
task_info = task_info.reset_index().drop(columns=["important", "match", "data"]) 
task_info["task_node"] = task_info["task_node"].astype(float)
task_info

,task_node,taskcol_name,task_params
0,7.0,"LED1(1,2)",[50.0]
1,7.0,"ASND(6,20)","[50.0, 1.0, 1.0, 40.0, 1000.0]"
2,7.0,"TTLP1(15,1)","[100.0, 1.0, 1.0]"
3,7.0,"TTLP2(15,2)",[40.0]
4,7.0,"TTLP3(15,3)","[50.0, 1.0, 1.0]"
5,8.0,"TTLP5(15,5)",[40.0]
6,9.0,"TTLP6(15,6)",[40.0]
7,11.0,"LK1(5,1)",[2.0]
8,12.0,"LED1(1,2)",[50.0]
9,12.0,"ASND(6,20)","[50.0, 1.0, 1.0, 40.0, 5000.0]"


In [7]:
event_channels_df = channels.merge(event_df, on=["family", "nbre"], how="right").merge(task_info, on=["taskcol_name", "task_node"], how="left").sort_values("t")
print(event_channels_df.to_string())

     channel_name  family  nbre taskcol_name  poly_evnum         t  _P  _V  _L  _R   _T   _W   _X   _Y  _Z  task_node                     task_params
0             NaN      10     1          NaN           0     0.006   1   1   0   0    1    0    0    0   0        1.0                             NaN
1             NaN      10     1          NaN           1     0.023   1   1   0   0   36    0    0    0   0       36.0                             NaN
2             PAD       6    22    PAD(6,22)           2     0.023   1   1   1   0    2    1    1    1   0       36.0                             NaN
3             NaN      10     1          NaN           3     0.118   1   1   0   0   38    0    0    0   0       38.0                             NaN
11           LED1       1     2    LED1(1,2)          11     0.128   0   0   0   0    0    0    0    0   0        3.0                             NaN
10           LED2       1     3    LED2(1,3)          10     0.128   0   0   0   0    0    0    0   

In [8]:
info = yaml.safe_load(info_path.open("r"))
info

{'processing': [{'duplicate_over': {'channel_name': ['LED(\\d+)',
     'ASND',
     'TTLP(1|3|4|8)']},
   'event_name': '{channel_name}',
   'method': 'output_binary_wave',
   'method_params': {'filter_expr': 'channel_name=="{channel_name}"',
    'state_expr': '_P',
    'count_expr': '_L'}},
  {'duplicate_over': {'channel_name': ['TTLP(2|5|6|7)']},
   'event_name': '{channel_name}',
   'method': 'input_binary_wave',
   'method_params': {'filter_expr': 'channel_name=="{channel_name}"',
    'state_expr': '_P'}},
  {'duplicate_over': {'channel_name': ['L(\\d+)']},
   'event_name': '{channel_name}',
   'method': 'input_binary_wave',
   'method_params': {'filter_expr': 'channel_name=="{channel_name}"',
    'state_expr': '_V'}},
  {'duplicate_over': {'side': ['_P', '_V']},
   'event_name': 'PAD{side}',
   'method': 'input_binary_wave',
   'method_params': {'filter_expr': 'channel_name=="PAD"',
    'state_expr': '{side}'}},
  {'duplicate_over': {'channel_name': ['LK(\\d+)']},
   'event_name':

In [9]:
event_spec = []

for item in info["processing"]:
    if "duplicate_over" not in item:
        item["duplicate_over"] = {}
    from helper import generate_duplicate_table, replace_vals
    duplication = generate_duplicate_table(item["duplicate_over"], dict(channel_name=channels["channel_name"]))
    for _, row in duplication.iterrows():
        final_d = replace_vals(item, row.to_dict())
        ev_name = final_d["event_name"]
        if "display" in info and "rename"in info["display"] and ev_name in info["display"]["rename"]:
            final_d["display_name"] = info["display"]["rename"][ev_name]
        else:
            final_d["display_name"] = ev_name
        del final_d["duplicate_over"]
        event_spec.append(final_d)

unique_df = pd.DataFrame(event_spec)["event_name"].value_counts().reset_index()
if not (unique_df["count"] == 1).all():
    display(unique_df.loc[unique_df["count"] > 1])
    raise Exception(f"Event name duplication")


display(pd.DataFrame(event_spec))
event_spec = {v["event_name"]: v for v in event_spec}


,event_name,method,method_params,display_name,metadata
0,LED1,output_binary_wave,"{'filter_expr': 'channel_name==""LED1""', 'state...",LED1,NaN
1,LED2,output_binary_wave,"{'filter_expr': 'channel_name==""LED2""', 'state...",LED2,NaN
2,LED3,output_binary_wave,"{'filter_expr': 'channel_name==""LED3""', 'state...",LED3,NaN
3,ASND,output_binary_wave,"{'filter_expr': 'channel_name==""ASND""', 'state...",ASND,NaN
4,TTLP1,output_binary_wave,"{'filter_expr': 'channel_name==""TTLP1""', 'stat...",cue,NaN
5,TTLP3,output_binary_wave,"{'filter_expr': 'channel_name==""TTLP3""', 'stat...",TTLP3,NaN
6,TTLP4,output_binary_wave,"{'filter_expr': 'channel_name==""TTLP4""', 'stat...",TTLP4,NaN
7,TTLP8,output_binary_wave,"{'filter_expr': 'channel_name==""TTLP8""', 'stat...",TTLP8,NaN
8,TTLP2,input_binary_wave,"{'filter_expr': 'channel_name==""TTLP2""', 'stat...",TTLP2,NaN
9,TTLP5,input_binary_wave,"{'filter_expr': 'channel_name==""TTLP5""', 'stat...",TTLP5,NaN


In [10]:
event_channels_df = event_channels_df.sort_values(["t", "poly_evnum"])
event_channels_df["used"]=False
from ipywidgets import widgets
outs = {k: display(display_id=f'{k}') for k in event_spec}
def compute_relevant(config):
    df = pd.DataFrame()
    ev_name = config["event_name"]
    df["t"] = event_channels_df["t"]
    df["task_params"] = event_channels_df["task_params"]
    df["task_node"] = event_channels_df["task_node"]
    for expr in ["filter_expr", "state_expr", "count_expr"]:
        if expr in config["method_params"]:
            df[expr.replace("_expr", "_value")] = event_channels_df.eval(config["method_params"][expr])
    if "metadata" in config:
        metadata = pd.DataFrame()
        for k, v in config["metadata"].items():
            metadata[k] = event_channels_df.eval(v)
        df["metadata"] = metadata.apply(lambda row: row.to_dict(), axis=1)
    else:
        df["metadata"] = [{}] * len(df.index)
    filtered = df.loc[df["filter_value"]] if "filter_expr" in config["method_params"] else df
    if "state_expr" in config["method_params"]:
        relevant = filtered.loc[filtered["state_value"] != filtered["state_value"].shift(1)].copy()
        relevant["duration"] = relevant["t"].shift(-1) - relevant["t"]
    else:
        relevant = filtered.copy()
    outs[config["event_name"]].update(relevant.rename_axis([ev_name]))
    return relevant



,t,task_params,task_node,filter_value,state_value,count_value,metadata,duration
LED1,,,,,,,,
8,0.128,NaN,3.0,True,0,0,{},1.088
23,1.216,[50.0],12.0,True,1,1,{},0.050
26,1.266,[50.0],12.0,True,0,1,{},31.002
70,32.268,NaN,25.0,True,1,1,{},2.000
75,34.268,NaN,17.0,True,0,0,{},3.374
...,...,...,...,...,...,...,...,...
5536,1139.026,[50.0],12.0,True,0,1,{},21.805
5569,1160.831,NaN,25.0,True,1,1,{},2.000
5582,1162.831,NaN,17.0,True,0,0,{},10.153


,t,task_params,task_node,filter_value,state_value,count_value,metadata,duration
LED2,,,,,,,,
7,0.128,NaN,3.0,True,0,0,{},32.140
69,32.268,NaN,25.0,True,1,1,{},2.000
74,34.268,NaN,17.0,True,0,0,{},32.543
317,66.811,NaN,49.0,True,1,1,{},2.000
326,68.811,NaN,17.0,True,0,0,{},1.159
...,...,...,...,...,...,...,...,...
5410,1111.460,NaN,17.0,True,0,0,{},23.337
5474,1134.797,NaN,25.0,True,1,1,{},2.000
5479,1136.797,NaN,17.0,True,0,0,{},24.034


,t,task_params,task_node,filter_value,state_value,count_value,metadata,duration
LED3,,,,,,,,
6,0.128,NaN,3.0,True,0,0,{},0.070
13,0.198,NaN,4.0,True,1,1,{},1.018
22,1.216,NaN,12.0,True,0,0,{},31.052
68,32.268,NaN,25.0,True,1,1,{},2.000
73,34.268,NaN,17.0,True,0,0,{},1.170
...,...,...,...,...,...,...,...,...
5640,1165.343,NaN,2.0,True,0,0,{},0.118
5649,1165.461,NaN,4.0,True,1,1,{},1.084
5658,1166.545,NaN,33.0,True,0,0,{},5.449


,t,task_params,task_node,filter_value,state_value,count_value,metadata,duration
ASND,,,,,,,,
21,1.216,"[50.0, 1.0, 1.0, 40.0, 5000.0]",12.0,True,1,1,{},0.050
25,1.266,"[50.0, 1.0, 1.0, 40.0, 5000.0]",12.0,True,0,1,{},36.376
136,37.642,"[50.0, 1.0, 1.0, 40.0, 5000.0]",12.0,True,1,1,{},0.050
140,37.692,"[50.0, 1.0, 1.0, 40.0, 5000.0]",12.0,True,0,1,{},8.374
270,46.066,"[50.0, 1.0, 1.0, 40.0, 1000.0]",41.0,True,1,1,{},0.050
...,...,...,...,...,...,...,...,...
5535,1139.026,"[50.0, 1.0, 1.0, 40.0, 5000.0]",12.0,True,0,1,{},27.519
5657,1166.545,NaN,33.0,True,1,1,{},0.300
5664,1166.845,NaN,33.0,True,0,1,{},6.139


,t,task_params,task_node,filter_value,state_value,count_value,metadata,duration
TTLP1,,,,,,,,
20,1.216,"[100.0, 1.0, 1.0]",12.0,True,1,1,{},0.096
27,1.312,"[100.0, 1.0, 1.0]",12.0,True,0,1,{},36.330
135,37.642,"[100.0, 1.0, 1.0]",12.0,True,1,1,{},0.101
142,37.743,"[100.0, 1.0, 1.0]",12.0,True,0,1,{},8.323
269,46.066,"[100.0, 1.0, 1.0]",41.0,True,1,1,{},0.100
...,...,...,...,...,...,...,...,...
5537,1139.076,"[100.0, 1.0, 1.0]",12.0,True,0,1,{},27.469
5656,1166.545,"[100.0, 1.0, 1.0]",33.0,True,1,1,{},0.100
5662,1166.645,"[100.0, 1.0, 1.0]",33.0,True,0,1,{},6.339


...

,t,task_params,task_node,filter_value,state_value,metadata,duration
LK2_reward,,,,,,,


,t,task_params,task_node,filter_value,state_value,metadata,duration
polytask_linechange,,,,,,,
0,0.006,NaN,1.0,True,1,{},0.017
1,0.023,NaN,36.0,True,36,{},0.095
3,0.118,NaN,38.0,True,38,{},0.010
4,0.128,NaN,2.0,True,2,{},0.000
5,0.128,NaN,3.0,True,3,{},0.070
...,...,...,...,...,...,...,...
5706,1172.983,NaN,41.0,True,41,{},0.149
5719,1173.132,NaN,42.0,True,42,{},0.175
5725,1173.307,NaN,43.0,True,43,{},0.200


,t,task_params,task_node,filter_value,state_value,metadata,duration
zone_change,,,,,,,
303,60.02,NaN,42.0,True,0,{},NaN


,t,task_params,task_node,filter_value,metadata
RD,,,,,


,t,task_params,task_node,filter_value,metadata
pause,,,,,
5749,1176.842,NaN,45.0,True,{'cause': 1}


In [11]:
from helper import add_to_error, json_merge
def compute_metadata(join_metadata, **warnings):
    kept_warnings = {}
    for k, v in warnings.items():
        vals = {v for v in v.values() if not pd.isna(v)}
        
        if len(vals) > 1:
            kept_warnings[k] = v
    if len(kept_warnings) > 0:
        metadata = json_merge(*join_metadata, dict(warnings=kept_warnings))
    else:
        metadata = json_merge(*join_metadata)
    return metadata


def output_accumulator_binary_wave(relevant: pd.DataFrame, config):
    res = []
    for _,grp in  relevant.groupby((relevant["state_value"] ==1).cumsum()):
        starts = grp[grp["state_value"]==1]
        if len(starts.index) ==0: continue
        elif len(starts.index) > 1: display(grp); raise Exception(f"Problem {len(starts.index)}")
        else:
            start = starts["t"].iat[0]
            rises = [0] + (grp["t"].loc[grp["state_value"] > grp["state_value"].shift(1)] -start).to_list()
            duration_on = config["method_params"]["duration"]
            expected_n_pulses = grp["task_params"].iat[0][0] 
            # if isinstance( grp["task_params"].iat[-1], list) else grp["task_params"].iat[-1]
            duration = rises[-1] + duration_on
            metadata = compute_metadata(grp["metadata"], n_rises_mismatch=dict(read=len(rises), expected_from_task=expected_n_pulses))
            res.append(dict(t=start, metadata=metadata, duration=[duration]*len(rises), rises=rises, duration_on=duration_on))
    return pd.DataFrame(res).assign(wave_type="binary")

def simple_binary_wave(relevant: pd.DataFrame, config):
    if relevant["state_value"].iat[0] == 0:
        relevant = relevant.iloc[1:, :].copy()
    if "count_value" in relevant.columns:
        relevant["next_count_value"] = relevant["count_value"].shift(-1)
    
    relevant["next_metadata"] = relevant["metadata"].shift(-1)
    rises = relevant.iloc[::2].copy()

    rises["corrected_duration"] = np.where(
        rises["duration"] <= 0.001, 
        rises["task_params"].apply(lambda l: l[0] if isinstance(l, list) and len(l) ==1 else np.nan), np.nan)
    rises["corrected_duration"] = rises["corrected_duration"].fillna(rises["duration"]).fillna(-1)

    rises["duration_on"] = rises["task_params"].apply(lambda l: float(l[0])/1000 if isinstance(l, list) and len(l) > 0 else np.nan).fillna(rises["corrected_duration"])
    rises["duration_off"] = rises["task_params"].apply(lambda l: float(l[1])/1000 if  isinstance(l, list) and len(l) > 1 else 0)
    rises["expected_n_pulses"] = rises["next_count_value"] if "count_value" in rises.columns else 1
    rises["cycle_duration"] = rises["duration_on"] + rises["duration_off"]
    rises["n"] = np.minimum(rises["expected_n_pulses"].fillna(1).astype(int), np.ceil(rises["corrected_duration"]/(rises["cycle_duration"])).astype(int))
    rises["rises"] = rises.apply(lambda row: [i * row["cycle_duration"] for i in range(row["n"])], axis=1)
    rises["metadata"] = rises.apply(lambda row: compute_metadata(
        [row["metadata"], row["next_metadata"]], 
        n_rises_mismatch=dict(read=len(row["rises"]), expected_from_task=row["expected_n_pulses"]),
        duration_correction=dict(read=row["duration"], corrected_using_task=row["corrected_duration"] if row["corrected_duration"] >=0 else np.nan)
    ), axis=1)
    rises["duration"] = np.where(rises["corrected_duration"]>=0, rises["corrected_duration"], np.nan)
    return rises[["t","metadata", "duration", "rises", "duration_on"]].assign(wave_type="binary").copy()



In [12]:



all = []
for ev_name, item in event_spec.items():
    relevant = compute_relevant(item)
    if len(relevant.index) ==0:
        continue
    if item["method"] == "output_accumulator_binary_wave":
        events = output_accumulator_binary_wave(relevant, item)
    elif item["method"].endswith("binary_wave"):
        events = simple_binary_wave(relevant, item)
    elif item["method"]=="step_wave":
        events = relevant[["t", "metadata","duration", "state_value"]].assign(wave_type="constant", prev_state_value=relevant["state_value"].shift(1), next_state_value=relevant["state_value"].shift(-1))
    elif item["method"]=="event":
        events = relevant[["t", "metadata"]].assign(wave_type=None, duration=None)
    events.insert(1, "event_name", ev_name)
    event_cols = ["t","event_name", "duration", "metadata"]
    events["waveform_info"] = events.apply(lambda row: {k:v for k, v in row.items() if not k in event_cols}, axis=1)
    events = events[event_cols + ["waveform_info"]]
    if len(events.index)!=0:
        all.append(events[[c for c in events.columns if events[c].count() > 0]])
all = pd.concat(all).sort_values("t")
display(all)
  

,t,event_name,duration,metadata,waveform_info
0,0.006,polytask_linechange,0.017,{},"{'state_value': 1, 'wave_type': 'constant', 'p..."
1,0.023,polytask_linechange,0.095,{},"{'state_value': 36, 'wave_type': 'constant', '..."
2,0.023,PAD_P,12.245,{},"{'rises': [0.0], 'duration_on': 12.24500000000..."
2,0.023,PAD_V,12.411,{},"{'rises': [0.0], 'duration_on': 12.411, 'wave_..."
3,0.118,polytask_linechange,0.01,{},"{'state_value': 38, 'wave_type': 'constant', '..."
...,...,...,...,...,...
5740,1174.757,PAD_P,0.358,{},"{'rises': [0.0], 'duration_on': 0.357999999999..."
5743,1174.940,PAD_V,-1.0,None,"{'rises': [-0.0], 'duration_on': -1.0, 'wave_t..."
5745,1175.145,PAD_P,-1.0,None,"{'rises': [-0.0], 'duration_on': -1.0, 'wave_t..."
5746,1175.902,LK1_lick,-1.0,"{'warnings': {'n_rises_mismatch': {'read': 0, ...","{'rises': [], 'duration_on': 0.002, 'wave_type..."


In [13]:
import json
all["metadata_json"] = all["metadata"].apply(lambda d: json.dumps(d) if not pd.isna(d) else "{}")
all["waveform_info_json"] = all["waveform_info"].apply(lambda d: json.dumps(d) if not pd.isna(d) else "{}")

all.drop(columns=["waveform_info", "metadata"]).to_csv(base.parent/ (base.name + "-- events.tsv"), sep="\t", index=False)

In [14]:
reloaded = pd.read_csv(base.parent/ (base.name + "-- events.tsv"), sep="\t", index_col=False)
reloaded["metadata"] = reloaded["metadata_json"].apply(lambda s: json.loads(s))
reloaded["waveform_info"] = reloaded["waveform_info_json"].apply(lambda s: json.loads(s))
reloaded.drop(columns=["waveform_info_json", "metadata_json"])

,t,event_name,duration,metadata,waveform_info
0,0.006,polytask_linechange,0.017,{},"{'state_value': 1, 'wave_type': 'constant', 'p..."
1,0.023,polytask_linechange,0.095,{},"{'state_value': 36, 'wave_type': 'constant', '..."
2,0.023,PAD_P,12.245000000000001,{},"{'rises': [0.0], 'duration_on': 12.24500000000..."
3,0.023,PAD_V,12.411,{},"{'rises': [0.0], 'duration_on': 12.411, 'wave_..."
4,0.118,polytask_linechange,0.010000000000000009,{},"{'state_value': 38, 'wave_type': 'constant', '..."
...,...,...,...,...,...
2841,1174.757,PAD_P,0.35799999999994725,{},"{'rises': [0.0], 'duration_on': 0.357999999999..."
2842,1174.940,PAD_V,-1.0,{},"{'rises': [-0.0], 'duration_on': -1.0, 'wave_t..."
2843,1175.145,PAD_P,-1.0,{},"{'rises': [-0.0], 'duration_on': -1.0, 'wave_t..."
2844,1175.902,LK1_lick,-1.0,"{'warnings': {'n_rises_mismatch': {'read': 0, ...","{'rises': [], 'duration_on': 0.002, 'wave_type..."
